In [1]:
import os
import numpy as np
import codecs
import random
from time import time()

In [3]:
# def get_all_sentences_from_txt(file_name):
#     sentences=[]
#     f = open(file_name, 'rt')
#     sentences_words=[]
#     for line in f:
#         sentences.append(line)
#     f.close()
#     return sentences

# def save_to_txt(sentences_array,file_output_name):
#     if os.path.isfile(file_output_name):
#         os.remove(file_output_name)
#     with open(file_output_name, "a") as a_file:
#         for elem in sentences_array:
#             a_file.write(elem)
#             a_file.write("\n")
#     a_file.close()

In [4]:
# train_words=get_all_sentences_from_txt("../dataset/raw/wiki_words_bitext_pos.txt")
# train_tags=get_all_sentences_from_txt("../dataset/raw/wiki_tags_bitext_pos.txt")

## Split Train, Valid and Test datasets

In [14]:
len_dataset=en(train_tags)
train_f=int(0.8*len_dataset)
print(train_f)
valid_f=int(0.9*len_dataset)
valid_f

1346363


1514658

In [ ]:
sed -n '1,1346363p;1346364q' wiki_words_bitext_pos.txt > ../train_valid/wiki_train_words_bitext_pos.txt
sed -n '1,1346363p;1346364q' wiki_tags_bitext_pos.txt > ../train_valid/wiki_train_tags_bitext_pos.txt

sed -n '1346364,1514658p;1514659q' wiki_words_bitext_pos.txt > ../train_valid/wiki_valid_words_bitext_pos.txt
sed -n '1346364,1514658p;1514659q' wiki_tags_bitext_pos.txt > ../train_valid/wiki_valid_tags_bitext_pos.txt

sed -n '1514659,1682954p' wiki_words_bitext_pos.txt > ../train_valid/wiki_test_words_bitext_pos.txt
sed -n '1514659,1682954p' wiki_tags_bitext_pos.txt > ../train_valid/wiki_test_tags_bitext_pos.txt

### Generate vocabularies ==> word, tag and tkt

##### Word vocab

In [17]:
%%time
#254294 rows
!onmt-build-vocab --size 1000000 --save_vocab ../dataset/train_valid/wiki-src-train-vocab.txt ../dataset/train_valid/wiki_train_words_bitext.txt

Wall time: 53.6 s


2020-08-09 23:23:10.248729: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'cudart64_101.dll'; dlerror: cudart64_101.dll not found
2020-08-09 23:23:10.250171: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2020-08-09 23:23:19.661410: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2020-08-09 23:23:19.663697: E tensorflow/stream_executor/cuda/cuda_driver.cc:351] failed call to cuInit: UNKNOWN ERROR (303)
2020-08-09 23:23:19.681906: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: LAPTOP-79FP3JFQ
2020-08-09 23:23:19.683028: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: LAPTOP-79FP3JFQ
2020-08-09 23:23:19.691300: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instruc

##### Target vocab

In [18]:
%%time
!onmt-build-vocab --size 1000 --save_vocab ../dataset/train_valid/wiki-tgt-train-vocab.txt ../dataset/train_valid/wiki_train_tags_bitext.txt

Wall time: 39.5 s


2020-08-09 23:24:08.397356: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'cudart64_101.dll'; dlerror: cudart64_101.dll not found
2020-08-09 23:24:08.397974: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2020-08-09 23:24:13.033383: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2020-08-09 23:24:13.034059: E tensorflow/stream_executor/cuda/cuda_driver.cc:351] failed call to cuInit: UNKNOWN ERROR (303)
2020-08-09 23:24:13.043040: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: LAPTOP-79FP3JFQ
2020-08-09 23:24:13.043899: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: LAPTOP-79FP3JFQ
2020-08-09 23:24:13.044743: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instruc

##### Tokenize Characters

In [19]:
%%time
!onmt-build-vocab --tokenizer_config ../config/char_tokenization.yml --size 10000 --save_vocab ../dataset/train_valid/wiki-src-train-tkt-vocab.txt ../dataset/train_valid/wiki_train_words_bitext.txt

Wall time: 1min 16s


2020-08-09 23:24:53.613459: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'cudart64_101.dll'; dlerror: cudart64_101.dll not found
2020-08-09 23:24:53.614928: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2020-08-09 23:24:57.252575: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2020-08-09 23:24:57.253148: E tensorflow/stream_executor/cuda/cuda_driver.cc:351] failed call to cuInit: UNKNOWN ERROR (303)
2020-08-09 23:24:57.261859: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: LAPTOP-79FP3JFQ
2020-08-09 23:24:57.263179: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: LAPTOP-79FP3JFQ
2020-08-09 23:24:57.263908: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instruc

In [ ]:
$ wc -l wiki*
   168296 wiki_test_tags_bitext.txt
   168296 wiki_test_words_bitext.txt
  1346363 wiki_train_tags_bitext.txt
  1346363 wiki_train_words_bitext.txt
   168295 wiki_valid_tags_bitext.txt
   168295 wiki_valid_words_bitext.txt
      189 wiki-src-train-tkt-vocab.txt
   447265 wiki-src-train-vocab.txt
       86 wiki-tgt-train-vocab.txt
  3813448 total
